<a href="https://colab.research.google.com/github/ewilson2023/History-Name-Mashup-Generator/blob/main/HistoryChalengeNameGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is this
A "fantasy" name generator. Picks syllables of names from Ancient Egypt, Ancient Greece, Ancient Rome, and Ancient Near East and generates a mashup name

## How to use it
1) if the table of contents isn't visible, click the hamburger button at the top of the left vertical menu bar.
2) Click "Main".
3) If on mobile, close the table of contents to see the code
4) At the top of the page, you'll see a menu with options for "Commands", "Code", "Text", and "Run all".
5) Click "Run all".
6) To generate again, click the play button under the "Main" label.

# Classes and Utilities

In [ ]:
# class for each cultur, storing sorted dictionaries

class Culture:
  def __init__(self, p, m, s, name):
    self.name = name
    self.pre = sort(p)
    self.mid = sort(m)
    self.suf = sort(s)




*   dicts lack explicit structure
* TypedDict: a fixed-shape dictionary where the keys are always the same bucket names, and each key always maps to the same kind of value


   

In [ ]:
from typing import TypedDict

# takes the form
# "name":   list["string"],
# "name":   list[int],
"""
SortBuckets is not a class at runtime. A SortedBuckets b
is a dictionary that has specific keys and value types.
b["all"] is a list of strings, b["c_any"] is a list of ints
"""
class SortBuckets(TypedDict):
    all: list[str]
    vv: list[int]; cc: list[int]
    vc: list[int]; cv: list[int]
    c: list[int];  v: list[int]
    c_any: list[int]; v_any: list[int]
    any_c: list[int]; any_v: list[int]

In [ ]:
# return a dictionary sorted by consonant/vowel ending
def sort(list: list[str]) -> dict:
    dict = {
          "all" :[],
          "vv": [], "cc": [],
          "vc": [], "cv": [],
          "c": [],  "v": [],

          "c_any": [], "v_any": [],
          "any_c": [], "any_v": [],
      }
    for idx, str in enumerate(list):
        dict["all"].append(str)

        str= str.lower()

        first = str[0]
        last = str[len(str) - 1]

        # if syllable is only one char
        if len(str) == 1:
          if isConsonant(first) and first != "y":
            dict["c"].append(idx)
            dict["cc"].append(idx)
            dict["c_any"].append(idx)
            dict["any_c"].append(idx)
          else:
            dict["v"].append(idx)
            dict["vv"].append(idx)
            dict["v_any"].append(idx)
            dict["any_v"].append(idx)
          continue

        # if the syllable start with a - or ', add to both "any" lists
        elif first == "-" or first == "'":
          dict["c_any"].append(idx)
          dict["v_any"].append(idx)

        # if syllable has 2 or more chars
        else:
          # if first char is a consonant or consonant-type y
          if (isConsonant(first) and first != "y") or (first == "y" and isVowel(str[1])):
            dict["c_any"].append(idx)

            if (isConsonant(last)) or (str[len(str)-2]+last) == "qu":
              dict["cc"].append(idx)
              dict["any_c"].append(idx)
            else:
              dict["cv"].append(idx)
              dict["any_v"].append(idx)
          else:
            dict["v_any"].append(idx)
            if isConsonant(last) or (str[len(str)-2]+last) == "qu":
              dict["vc"].append(idx)
              dict["any_c"].append(idx)
            else:
              dict["vv"].append(idx)
              dict["any_v"].append(idx)


    return dict


In [ ]:
def isVowel(char):
  char = char.lower()
  vowels = ["a", "e", "i", "o", "u", "ë"]

  if char in vowels:
    return True
  else:
    return False

def isConsonant(str):
  return not isVowel(str)

In [ ]:
  """ Pick an index into b["all"], using fallback logic:
      1) try the preferred index-bucket (e.g., "c_any")
      2) if empty, try the other bucket (e.g., "v_any")
      3) if still empty, fall back to ANY valid index from b["all"]
    Returns: an int index suitable for b["all"][idx] """
def pick_idx(
      b: SortBuckets, prefer: str, other: str) -> int:
        pool: list[int] = b[prefer]     # ex. b = [0, 5, 8] (idxs)
        if not pool:                    # if empty, pick from other list of idxs
            pool = b[other]             # still a list[int]
        if not pool:                    # if still empty, pick from all
            pool = list(range(len(b["all"])))
        return random.choice(pool)      # return random idx int from the pool

def pick_syllable(
      cult: Culture,
      next: str, part_choice: str ) -> str:
    sbl: str
    idx: int

    if next!= 'c' and next!= 'v':
      raise ValueError("next must be 'c' or 'v'")

    # Corrected logic: if previous ended in Vowel (next='c'), current should start with Consonant (c_any)
    # If previous ended in Consonant (next='v'), current should start with Vowel (v_any)
    if next == 'c':
      syl_type = "c_any"
    else: # next == 'v'
      syl_type = "v_any"

    if part_choice == "prefix":
      word_part = cult.pre
    elif part_choice == "middle":
      word_part = cult.mid
    else: # part == "suffix"
      word_part = cult.suf


    # choose what pool to get the index from
    while word_part[syl_type] == []:
        cult = random.choice(all_cultures)
        if part_choice == "prefix":
          word_part = cult.pre
        elif part_choice == "middle":
          word_part = cult.mid
        else: # part == "suffix"
          word_part = cult.suf

    syls_of_type = word_part.get(syl_type)
    all = word_part.get("all")

    idx = random.choice(syls_of_type)


    # choose the syllable associated with this index
    syllable = word_part.get("all")[idx]

    # if a prefix was chosen in fallback, make the
    # first letter lowercase (hypen aware)
    non_hyph_upper: bool = syllable[0].isalpha() and syllable[0].isupper()

    if part_choice != "prefix" and non_hyph_upper:
      syllable = syllable[0].lower() + syllable[1:]

    result: dict = {"syl": syllable, "cult": cult}

    return result


In [ ]:
import random

##################### v2
def generateName(cultures: list[Culture]) -> None:

  """############### PREFIX ###############"""
  name: str
  cult: Culture = random.choice(cultures)

  # prefix: random of all this cultures's prefixes
  p: str = random.choice(cult.pre.get("all"))

  info1: str = (cult.name)
  info2: str = (p)

  name = p;   """add it to name"""

  if isVowel(p[len(p) - 1]):
    next = 'c'
  else:
    next = 'v'

  """############### MIDDLE ###############"""
  m: str = ""
  if random.choice([True, False]):
    # choose a new culture
    cult = random.choice(cultures)

    pickSyl = pick_syllable(cult, next, "middle")

    m = pickSyl.get("syl")

    if m != "":
      if isVowel(m[len(m) - 1]):
        next = 'c'
      else:
        next = 'v'

      info1 = (info1+ " — " + pickSyl.get("cult").name)
      info2 = (info2+ " — " + m)

      """add it to name"""
      if not name[len(name) - 1].isalpha() and not m[0].isalpha():
        name = name + str(m[1:])
      else:
        name = name + m

    # 25% chance of second mid syllable, same culture
    if random.choice([True, False]):
      pickSyl = pick_syllable(cult, next, "middle")

      m = pickSyl.get("syl")

      if m != "" and len(name) + len(m) < 8:
        if isVowel(m[len(m) - 1]):
          next = 'c'
        else:
          next = 'v'

        info1 = (info1+ " — " + pickSyl.get("cult").name)
        info2 = (info2+ " — " + m)

        """add it to name"""
        if not name[len(name) - 1].isalpha() and not m[0].isalpha():
          name = name + str(m[1:])
        else:
          name = name + m


  """############### SUFFIX ###############"""
  # choose a new culture
  cult = random.choice(cultures)

  pickSyl = pick_syllable(cult, next, "suffix")
  s = pickSyl.get("syl")

  """add it to name"""
  if s != "":

    info1 = (info1+ " — " + cult.name)
    info2 = (info2+ " — " + s)

    """add it to name"""
    if not name[len(name) - 1].isalpha() and not s[0].isalpha():
      name = name + str(s[1:])
    else:
      name = name + s



  """############### OUTPUT ###############"""
  print(name)
  print("  "+info1)
  print("  "+info2 + "\n")


In [ ]:
#  syllables for all cultures
all = {
  #### EGYPTIAN ####
  "e_p": [
      "A","Nu", "Ha", "Dje", "Djo", "Amen", "Amon", "Anu", "Bas", "Hat",
      "Sob", "Nef", "Nefer", "Sen", "Ahm", "Akhe", "Khu", "Kh", "Sut", "Ka-",
      "Khn", "Kha", "Ram", "Thut", "Sme", "Set", "Pa-", "Aa", "Aam", "Hat-",
      "Dsj", "Dsja", "Osir", "Isi", "Ankh", "Ar", "Nub", "He"
    ],

  "e_m": [
      "ph", "phth", "ho", "huti", "nou", "shep", "kn", "-sch", "-nefer-",
      "de", "phr", "khm", "kh", "dj", "mh", "khk", "er", "utme", "-at-", "-en-",
      "hm", "khe", "nemh", "khn", "fer", "f", "pta", "djm", "fer", "-sa",
      "auten", "nkhk", "utmose", "ankh", "ankha", "kham", "-na", "khur-",
       "nefer", "khma", "khe", "ekhma", "khs", "khsep", "-anh"
    ],

  "e_s": [
      "are", "eru", "ys", "ek","esu", "et", "em", "at", "ten", "aa", "mat",
      "eb", "ut", "uti", "hotep", "ho", "tep", "are", "aten", "-hat", "i",
      "tari", "ti", "fre", "fre", "tem", "fra", "mose", "mon", "ket", "nofre",
      "kh", "det", "tep", "th", "nut", "sis", "mheb", "nkh", "-ap", "tef",
      "ankh", "ptah","met", "kmet", "djmet", "bek", "feru", "khamon", "khons"
      "sret", "sobek", "khufu", "psut", "phris", "-heb", "-i", "mhe", "-pu",
      "pet", "het", "pat", "-shen", "-hapi"
    ],

  #### ROMAN ####
  "r_p": [
      "Luc", "Val", "Mar", "Ovi", "Gai", "Hel", "Agrip", "Opp", "Mall",
      "C", "Tac", "Ta", "Ae", "Au", "Aure", "Cae", "Cl", "Fla", "Ru", "Lu",
      "Gl", "Gn", "Iu", "Ju", "Jul", "Liv", "Ma", "Marc", "St", "Men",
      "Max", "Maxi", "Por", "Prisc", "Quint", "Sci", "Sev", "Io", "Publ",
      "Silv", "Tit", "Va", "Vi", "Val", "Var", "Vib", "Cor", "Gall", "Ha",
      "P", "Tre", "Dru", "Septi", "Arca", "Can", "Lae", "Quatr", "Qu", "Qui",
      "Quint", "Ia", "Iac", "Flav", "Calv", "E", "Ter", "Sci", "Cre", "Cr",
      "Oct", "Octa", "Serv", "Octo", "Kae", "Jun", "Tut", "Tibur", "Tri",
      "Deci", "Decim", "Pae", "Sae", "Iust", "Bae", "Arc", "Arcad", "Arca",
      "Publi", "Sext", "Cocc", "Tert", "Secund", "Mae", "D", "Pela"
    ],
  "r_m": [
      "an", "in", "ian", "ae", "an", "en", "ar", "qu", "v", "fr", "aeb", "ill",
      "c", "n", "sc", "t", "p", "aia", "l", "lian", "mil", "qui", "ll", "vern",
      "ss", "ssian", "ian", "aud", "au", "ci", "m", "iab", "ael", "Nov", "nian"
      "up", "rc", "nl", "cell", "ae", "ver", "lp", "rqu", "quin", "ptim",
      "nti", "nt", "ell", "ni", "arat", "pst", "cl", "clect", "eia", "mir",
      "cret", "lla", "atin", "tor", "sent", "cil", "lian", "ian", "at", "spi",

    ],
  "r_s": [
      "ia", "ius", "us", "ina", "ella", "a", "iana", "aeus", "ars", "io",
      "ian", "ianus", "allas", "llus", "icus", "ilius", "eius", "sus", "sca",
      "ntia", "ata", "mus", "aeus", "llo", "llia", "llius", "aius", "spex",
      "eria", "erio", "cia", "tia", "tio", "cio", "cius", "ilina", "ssus"
    ],

  #### GREECE ####
  "g_p": [
      "Lys", "Kal", "Ari","Rho", "Kor", "Arte", "Ae", "Ch", "Chl",
      "Deme", "Dem", "Tel", "Tele", "Thou", "Eu", "A", "Tha", "Da",
      "Her", "Hesp", "Ny", "Ph", "Per", "Pel", "Phi", "Th", "Theo",
      "Tha", "Xa", "Xe", "Xeno", "Zo", "Ze", "Louk",
    ],
  "g_m": [
      "er", "in", "th", "t", "kr", "d", "tr", "k", "ll",
      "ph", "gor", "ch", "nt", "sch", "l", "ga", "pe", "ng", "ei",
      "el", "gath", "x", "xan", "lk", "ia", "mp", "el", "rk",
      "ari", "sth", "ian", "nth", "pho", "phe" "sthe", "hem", "rg"
      "thy", "pp", "mach", "ymph", "mph", "meni", "ych", "ou",
    ],
  "g_s": [
      "ius", "ias", "is", "as", "ache", "one", "is", "des", "os",
      "ios", "us", "es", "eas", "eus", "ate", "ates", "ice", "sos",
      "os", "oë", "eus", "sia", "kles", "ion", "e", "ton", "eon",
      "phon", "es", "d", "od", "iod", "ene", "ates", "kles", "che",
    ],

  #### NEAR EAST ####
  "ne_p": [
      "Ha", "Hamm", "Has", "Nab", "Nabo", "Nabu", "Nebu", "Esar", "E",
      "Bel", "Sar", "Sha", "Shal", "En", "Enh", "Me", "Tia", "Mesi",
      "Sin-", "A", "Lam", "Asor", "Ashur", "Lars", "Ea-", "Geme", "Iaa",
      "Ari", "I", "Baa", "Ish", "Isht", "Ensh", "Ara", "Uba", "Gem",
      "Ishm", "Ipqu-", "Ar'", "Aha", "Ku-", "Ga", "Ni", "Nii", "Enhe"
    ],

  "ne_m": [
      "mmu", "dru", "apla", "pol", "mmur", "nn", "zz", "mil", "nezz", "'ii",
      "sharra", "sharr", "sha", "sh", "kk", "ira", "ban", "shur", "ch", "mki",
      "dn", "adn", "lm", "man", "a", "shl", "nhe", "dd", "rh", "shm", "u", "i",
      "hati-", "-ban", "shkir", "di-", "-Sharr", "kha", "-na", "zi", "pqu-",
      "shtu", "z", "methr", "ii", "-Gam", "e", "ata", "dugg", "unz", "shtu",
      "quu", "gsumm", "urza", "hun", "-Wa", "qr", "shti-", "shkiri", "isht",
      "bni-", "ww", "'siu", "uiq", "qa", "iiarqu", "rqus", "zur-", "-Ka",
      "ssunu", "-Aya", "kiri", "kiti", "k"
    ],

  "ne_s": [
      "ssar", "assar", "usur", "nna", "car", "ar", "zzar", "rdoch", "it",
      "don", "rri", "i", "nezzar", "ton", "gon", "mis", "amis", "lshu", "shu",
      "bi", "neser", "mat", "at", "ltum", "tum", "rra", "shua",
      "itum", "na", "-ea", "-idi", "-eil", "-dan", "shi", "-nasir", "paa", "esi",
      "eesu", "-Aya", "kusu", "bitu", "lat", "aat", "ggat", "aa", "unu", "annu",
      "ka", "uu", "kir", "lnu", "qrat", "llat", "qqa", "rqusu", "qusu", "uun",
      "ti", "duana", "ana", "uana", "iti", "kaa",
    ]
}


# create class instance for each culture
Egypt = Culture(all.get("e_p"), all.get("e_m"), all.get("e_s"), "Egyptian")
Rome = Culture(all.get("r_p"), all.get("r_m"), all.get("r_s"), "Roman")
Greece = Culture(all.get("g_p"), all.get("g_m"), all.get("g_s"), "Greek")
Near_East = Culture(all.get("ne_p"), all.get("ne_m"), all.get("ne_s"), "Near East")

# list of all cultures/classes
all_cultures = [Egypt, Rome, Greece, Near_East]


# Main
AFTER pressing "Run All", press the play button below.

In [ ]:
# <- Hover over me!! Click that button!!

for i in range(5):
  generateName(all_cultures)

# hide
don't worry about this

In [ ]:
debug: bool = False
if debug:
  cult = Rome

  print(cult.name)

  pickSyl = pick_syllable(cult, 'v', "suffix")

  print("  "+ pickSyl.get("syl") + " - " + pickSyl.get("cult").name)

In [ ]:
debug: bool = False

def print_test(cult, word_part, syl_type):

  syls_of_type = word_part.get(syl_type)
  all = word_part.get("all")

  print(cult.name+" - "+str(len(syls_of_type))+"")
  # print("  "+str(all))

  result: str= ""
  for i, idx in enumerate(syls_of_type):
    result = result + ", "+ all[idx]

  result = result.removeprefix(", ")
  result = "  [" + result + "]"
  print(result)
  print()

if debug:
  for cult in all_cultures:
    word_part = cult.mid
    syl_type = "cc"
    print_test(cult, word_part, syl_type)
